In [1]:
import numpy as np
import gzip
import pickle
import collections
from scipy.stats import rankdata

In [3]:
w = np.random.rand(4)
# ne pas utiliser argsort
r = rankdata(w).astype(np.long) - 1
print(w)
print(r)
print(w[r])

[0.67804075 0.59569376 0.56421464 0.93670562]
[2 1 0 3]
[0.56421464 0.59569376 0.67804075 0.93670562]


/tmp/ipykernel_14126/1605382207.py:3: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  r = rankdata(w).astype(np.long) - 1


In [6]:
# chargement des données 

# Il faut au préalable télécharger les données ici :
# https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz
# mettez l'emplacement de votre fichier
f = gzip.open('/home/myriem/Documents/M1-ISD/proba_stat/TP5/mnist.pkl.gz', 'rb')
u = pickle._Unpickler(f)
u.encoding = 'latin1'
p = u.load()
# chaque set contient deux éléments qui sont tous les deux des array numpy:
# les images et les labels (label = le nombre écrit sur l'image)
train_set, valid_set, test_set = p
images, etiquettes = train_set
# les images sont des taille 28x28 pixels
dim = 28

In [8]:
X_test = test_set[0]
Y_test = test_set[1]

X_train = train_set[0]
Y_train = train_set[1]

X_valid = valid_set[0]
Y_valid = valid_set[1]

In [9]:
x = X_test[0]
y = Y_test[0]

print(x.shape, y.shape)

(784,) ()


In [21]:
X_test.shape

(10000, 784)

# Classificateur plus proche voisin simple

In [13]:
x = X_train[83]
y = Y_train[83]

i_min = -1
i_min_dist = float('inf')
#for i in range(X_train.shape[0]):
#    xi = X_train[i]
#    dist = np.sum((xi - x)**2)
#    if dist < i_min_dist:
#        i_min_dist = dist
#        i_min = i
#print(Y_train[i_min])



dist = np.sum((X_train - x) ** 2, axis=1)
i_min = np.argmin(dist)
print(Y_train[i_min])


class NNClassifier:
    def __init__(self, X_train, Y_train):
        self.X_train = X_train
        self.Y_train = Y_train
        
    def predict(self, x):
        dist = np.sum((self.X_train - x) ** 2, axis=1)
        i_min = np.argmin(dist)
        return self.Y_train[i_min]
    
    def accuracy(self, X, Y):
        n_correct = 0
        for i in range(X.shape[0]):
            n_correct += Y[i] == self.predict(X[i])
        return n_correct / X.shape[0]

6


In [17]:
c = NNClassifier(X_train, Y_train)
c.accuracy(X_train[:500], Y_train[:500])

1.0

In [19]:
c.accuracy(X_test[:500], Y_train[:500])

0.104

# Classificateur k-plus proche voisin

In [15]:
class KNNClassifier:
    def __init__(self, X_train, Y_train):
        self.X_train = X_train
        self.Y_train = Y_train
        
    def predict(self, x):
        dist = np.sum((X_train - x) ** 2, axis=1)
        r = rankdata(dist).astype(np.long) - 1
        knn = self.Y_train[r < 10]

        c = collections.Counter(knn)
        #pred, _ = max(c.items(), key=lambda i: i[1])
        pred = c.most_common()[0]
        return pred
    
    def accuracy(self, X, Y):
        n_correct = 0
        for i in range(X.shape[0]):
            n_correct += Y[i] == self.predict(X[i])
        return n_correct / X.shape[0]

In [16]:
k = 10
c = KNNClassifier(X_train, Y_train)
c.accuracy(X_train[:500], Y_train[:500])

/tmp/ipykernel_14126/1493182723.py:8: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  r = rankdata(dist).astype(np.long) - 1


array([0.972, 0.014])

In [ ]:
x = X_train[83]
y = Y_train[83]



# Classificateur "image moyenne"

In [27]:
class MeanClassifier:
    def __init__(self, l, X_train, Y_train):
        self.theta = np.empty((l, X_train.shape[1]))
        for i in range(l):
            self.theta[i] = np.mean(X_train[Y_train == i], axis=2)
        
    def predict(self, X):
        score = np.sum((X[:, None, :] - self.theta[None, :, :])**2, axis=2)
        return np.argmin(score, axis=1)
    
    def accuracy(self, X, Y):
        n_correct = 0
        for i in range(X.shape[0]):
            n_correct += Y[i] == self.predict(X[i])
        return n_correct / X.shape[0]

In [28]:
c = MeanClassifier(10, X_train, Y_train)
c.accuracy(X_train[:500], Y_train[:500])

AxisError: axis 2 is out of bounds for array of dimension 2

In [20]:
X = X_test

X[:, None, :].shape

(10000, 1, 784)

In [ ]:
X = X_test

score = np.sum(X[:, None, :] - c.theta[None, :, :])**2; axis=2)
score